In [1]:
from collections import defaultdict
import tensorflow as tf
import numpy as np
import tflearn
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 10)
import cv2
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import tifffile as tiff
import sys
import sklearn
import time
import PIL
from scipy import misc

In [2]:
X_train = np.load("/data/zonghua/satellite/Final/Step_1_Prediction/my_data/train_data.npy", 'r')
y_train = np.load("/data/zonghua/satellite/Final/Step_1_Prediction/my_data/train_label.npy", 'r')
X_test  = np.load("/data/zonghua/satellite/Final/Step_1_Prediction/my_data/part1_down_2400_as_test_data.npy", 'r')
y_test  = np.load("/data/zonghua/satellite/Final/Step_1_Prediction/my_data/part1_down_2400_as_test_label.npy", 'r')
y_train = 1. - y_train
y_test = 1. - y_test

In [3]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((48238, 112, 112, 8),
 (48238, 112, 112, 1),
 (11924, 112, 112, 8),
 (11924, 112, 112, 1))

In [2]:
def bn_relu_conv_concat(bottom, training, name, filters=12, stride=(1, 1), activation=tf.nn.relu):
    with tf.variable_scope("dense_layer_{}".format(name)):
        net = tf.layers.batch_normalization(bottom, training=training, name='bn_dense_{}'.format(name))
        net = activation(net, name='relu_dense_{}'.format(name))
        net = tf.layers.conv2d(net, filters, (3, 3), strides=stride, activation=None, padding='same', name='conv_dense_{}'.format(name))
        net = tf.concat([net, bottom], axis=3, name='concat_dense_{}'.format(name))
        return net

def transition(bottom, training, name, stride=(1, 1), activation=tf.nn.relu, pool=True):
    with tf.variable_scope("transition_layer_{}".format(name)):
        filters = bottom.get_shape().as_list()[3]
        net = tf.layers.batch_normalization(bottom, training=training, name='bn_transition_{}'.format(name))
        base = activation(net, name='relu_transition_{}'.format(name))
        net = tf.layers.conv2d(base, filters, (1, 1), strides=stride, activation=None, padding='same', name='conv_transition_{}'.format(name))
        if pool:
            net = tf.layers.average_pooling2d(net, (2, 2), strides=(2, 2), name="pool_{}".format(name))
        return base, net

def upsampling_2D(tensor, name, size=(2, 2)):
    H, W, _ = tensor.get_shape().as_list()[1:]

    H_multi, W_multi = size
    target_H = H * H_multi
    target_W = W * W_multi

    return tf.image.resize_nearest_neighbor(tensor, (target_H, target_W), name="upsample_{}".format(name))

def up_sample_conv(net, filters, strides, training, name, activation=tf.nn.relu):
    with tf.variable_scope("up_sample_{}".format(name)):
        net = tf.layers.conv2d(net, filters, (1, 1), activation=activation, padding="same", name="up_conv")
        if strides != 1:
            net = upsampling_2D(net, name, size=(strides, strides))
        with tf.variable_scope("conv_1"):
            conv_1 = tf.layers.conv2d(net, 1, (3, 3), activation=None, padding="same", name="conv")
            net = tf.layers.batch_normalization(conv_1, training=training, name="bn")
            net = activation(net, name="relu")
        with tf.variable_scope("conv_2"):
            conv_1 = tf.layers.conv2d(net, 1, (3, 3), activation=None, padding="same", name="conv")
            net = tf.layers.batch_normalization(conv_1, training=training, name="bn")
            net = activation(net, name="relu")
        net = tf.layers.conv2d(net, 1,  [1, 1], activation=None, name="Final")
        return net

def dense_net(X, training):
    with tf.variable_scope("Preprocessing"):
        net = X * 2 - 1
        net = tf.layers.conv2d(net, 8, (1, 1), name="color_space1_adjust")
    dsn1 = tf.layers.conv2d(net, 16, (7, 7), padding='same', name='first_conv', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(dsn1, 3, strides=2, padding='same', name='first_pool')
    
    with tf.variable_scope("dense_block_1"):
        for i in range(0, 6):
            net = bn_relu_conv_concat(net, training, i)
            
    dsn2, net = transition(net, training, 101)
    
    with tf.variable_scope("dense_block_2"):
        for i in range(8, 20):
            net = bn_relu_conv_concat(net, training, i)
            
    dsn3, net = transition(net, training, 102)
    
    with tf.variable_scope("dense_block_3"):
        for i in range(20, 48):
            net = bn_relu_conv_concat(net, training, i)
            
    dsn4, net = transition(net, training, 103)
    
    with tf.variable_scope("dense_block_4"):
        for i in range(48, 64):
            net = bn_relu_conv_concat(net, training, i)
            
    dsn5, net = transition(net, training, 104)
    
    return dsn1, dsn2, dsn3, dsn4, dsn5
    
def make_hed(X, training):
    dsn1, dsn2, dsn3, dsn4, dsn5 = dense_net(X, training)
    
    dsn1 = up_sample_conv(dsn1, 16, 1, training, "1")
    dsn2 = up_sample_conv(dsn2, 32, 2, training, "2")
    dsn3 = up_sample_conv(dsn3, 64, 4, training, "3")
    dsn4 = up_sample_conv(dsn4, 128, 8, training, "4")
    dsn5 = up_sample_conv(dsn5, 256, 16, training, "5")
    
    dsn = tf.concat([dsn1, dsn2, dsn3, dsn4, dsn5], axis=3, name='concat')
    dsn = tf.reshape(dsn, [-1, 112, 112, 5])
    dsn = tf.layers.conv2d(dsn, 1, (1, 1), name='output', activation=None, padding='same')

    return dsn1, dsn2, dsn3, dsn4, dsn5, dsn

def IOU_loss(y_pred, y_true):
    y_pred = tf.round(1. - y_pred)
    y_true = tf.round(1. - y_true)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.cast(y_true, tf.float32)
    
    H, W, _ = y_pred.get_shape().as_list()[1:]
    pred_flat = tf.reshape(y_pred, [-1, H * W])
    true_flat = tf.reshape(y_true, [-1, H * W])
    intersection = 2 * tf.reduce_sum(pred_flat * true_flat, axis=1) + 1e-12
    denominator = tf.reduce_sum(pred_flat, axis=1) + tf.reduce_sum(true_flat, axis=1) + 1e-7

    return tf.reduce_mean(intersection / denominator)

def IOU_no_classified(y_pred, y_true):
    y_pred = 1. - y_pred
    y_true = 1. - y_true
    
    H, W, _ = y_pred.get_shape().as_list()[1:]
    pred_flat = tf.reshape(y_pred, [-1, H * W])
    true_flat = tf.reshape(y_true, [-1, H * W])
    intersection = 2 * tf.reduce_sum(pred_flat * true_flat, axis=1) + 1e-12
    denominator = tf.reduce_sum(pred_flat, axis=1) + tf.reduce_sum(true_flat, axis=1) + 1e-7

    return tf.reduce_mean(intersection / denominator)

def class_balanced_sigmoid_cross_entropy(y_true, y_pred):
    y = tf.cast(y_true, tf.float32)
    
    count_neg = tf.reduce_sum(1. - y) + 1
    count_pos = tf.reduce_sum(y) + 1
    beta = count_neg / (count_neg + count_pos + 2)
    
    pos_weight = beta / (1 - beta)
    cost = tf.nn.weighted_cross_entropy_with_logits(y, y_pred, pos_weight)
    cost = tf.reduce_mean(cost * (1 - beta))
    
    return cost

In [3]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement=True
sess = tf.Session(config=config)
with tf.device("/gpu:3"):
    global_step_tensor = tf.Variable(1, trainable=False, name='global_step')
    with tf.variable_scope("X_Input_Layer"):
        X_layer = tf.placeholder(shape=[None, 112, 112, 8], dtype=tf.float32, name="X_layer")
    with tf.variable_scope("Y_Input_Layer"):
        y_layer = tf.placeholder(shape=[None, 112, 112, 1], dtype=tf.float32, name="y_layer")
    with tf.variable_scope("Mode_Layer"):
        trainning_mode = tf.placeholder(tf.bool, name="Mode")
    d1, d2, d3, d4, d5, pred_not_sigmoid = make_hed(X_layer, trainning_mode)
    pred = tf.nn.sigmoid(pred_not_sigmoid)
    tf.summary.image("Predicted_Mask", pred)
    X_layer_2015 = X_layer[:, :, :, :3]
    X_layer_2017 = X_layer[:, :, :, 4:7]
    tf.summary.image("2015", X_layer_2015)
    tf.summary.image("2017", X_layer_2017)
    tf.summary.image("Label", y_layer)
    
    with tf.variable_scope("Accuracy"):
        pred_int = tf.cast(tf.round(pred, name="round_pred"), dtype=tf.int32, name="cast_pred")
        y_int = tf.cast(tf.round(y_layer, name="round_y"), dtype=tf.int32, name="cast_y")
        acc = tf.contrib.metrics.accuracy(y_int, pred_int)
        tf.summary.scalar("Accuracy", acc)
        
    with tf.variable_scope("Loss"):
        loss = class_balanced_sigmoid_cross_entropy(y_layer, pred_not_sigmoid)
        tf.summary.scalar("Loss", loss)
    
    with tf.variable_scope("IOU"):
        IOU_op = IOU_loss(pred, y_layer)
        tf.summary.scalar("IOU", IOU_op)
        
    with tf.variable_scope("IOU_no_classified"):
        IOU_op_no_class = IOU_no_classified(pred, y_layer)
        tf.summary.scalar("IOU_no_classified", IOU_op_no_class)
        
    summary_op = tf.summary.merge_all()
    
    with tf.variable_scope("Test_Summary_INPUT"):
        test_IOU_input = tf.placeholder(dtype=tf.float32, name="Summary_INPUT_IOU")
        test_IOU_no_class_input = tf.placeholder(dtype=tf.float32, name="Summary_INPUT_IOU_NO_CLASSIFY")
        test_acc_input = tf.placeholder(dtype=tf.float32, name="Summary_INPUT_Acc")
        test_loss_input = tf.placeholder(dtype=tf.float32, name="Summary_INPUT_Loss")
    test_summary_IOU_op = tf.summary.scalar("epoch/IOU", test_IOU_input)
    test_summary_IOU_op_no_classified = tf.summary.scalar("epoch/IOU_no_classified", test_IOU_no_class_input)
    test_summary_acc_op = tf.summary.scalar("epoch/Accuracy", test_acc_input)
    test_summary_loss_op = tf.summary.scalar("epoch/Loss", test_loss_input)

    lr = tf.placeholder(tf.float32, name='Learning_Rate')
    choose_momentum = tf.placeholder(tf.int32, name='Choose_Optimizer')
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        global_step = tf.train.get_global_step()
        if choose_momentum == 1:
            train_step = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9).minimize(loss, global_step=global_step)
        else:
            train_step = tf.train.AdamOptimizer().minimize(loss, global_step=global_step)

In [9]:
!rm -rf tmp/Dense_HED/log_train
!rm -rf tmp/Dense_HED/log_test

In [10]:
train_summary_writer = tf.summary.FileWriter("tmp/Dense_HED/log_train", sess.graph)
test_summary_writer  = tf.summary.FileWriter("tmp/Dense_HED/log_test", sess.graph)

In [11]:
sess.run(tf.global_variables_initializer())
tf.train.global_step(sess, global_step_tensor)

1

In [12]:
def random_augmentation(imges, labels):
    au_images = []
    au_labels = []
    for i in range(len(imges)):
        img_tmp = np.copy(imges[i])
        label_tmp = np.copy(labels[i])
        rand_flip = np.random.randint(0, 4)
        rand_rotat = np.random.randint(0, 4)
        rand_change = np.random.randint(0, 2)
        if rand_flip != 3:
            img_tmp = cv2.flip(img_tmp, rand_flip - 1)
            label_tmp = cv2.flip(label_tmp, rand_flip - 1)
        M = cv2.getRotationMatrix2D((56, 56), rand_rotat*90, 1.0)
        img_tmp = cv2.warpAffine(img_tmp, M, (112, 112))
        label_tmp = cv2.warpAffine(label_tmp, M, (112, 112))
        if rand_change == 1:
            mean1 = np.mean(img_tmp[:, :, :4])
            mean2 = np.mean(img_tmp[:, :, 4:])
            rand_std1 = np.random.ranf() * 0.4 + 0.8
            rand_std2 = np.random.ranf() * 0.4 + 0.8
            rand_mean1 = (np.random.ranf() * 0.4 - 0.2) * mean1
            rand_mean2 = (np.random.ranf() * 0.4 - 0.2) * mean2
            img_tmp[:, :, :4] = (img_tmp[:, :, :4] * rand_std1 + rand_mean1).clip(0, 1)
            img_tmp[:, :, 4:] = (img_tmp[:, :, 4:] * rand_std2 + rand_mean2).clip(0, 1)
        au_images.append(img_tmp)
        au_labels.append(label_tmp)
    au_images = np.asarray(au_images)
    au_labels = np.asarray(au_labels).reshape(-1, 112, 112, 1)
    return au_images, au_labels

In [22]:
batch_size = 32
start_time = time.time()
learning_rate = 0.0001
change_step = 16
momentum_optimizer = 1
adam_step = 0

for i in range(108, 300):
    train_IOU = 0.0
    train_IOU_no_classified = 0.0
    train_accuary = 0.0
    train_loss = 0.0
    rand_indexes = np.random.choice(len(X_train), len(X_train), replace = False)
    for j in range(0, len(X_train), batch_size):
        X_batch, y_batch = random_augmentation(X_train[rand_indexes[j: j + batch_size]], y_train[rand_indexes[j: j + batch_size]])
        _, step_iou, step_iou_no_classified, step_acc, step_loss, step_summary, step_value = sess.run(
            [train_step, IOU_op, IOU_op_no_class, acc, loss, summary_op, global_step],
            feed_dict = {X_layer: X_batch, y_layer: y_batch, trainning_mode: True,\
                         lr: learning_rate, choose_momentum: momentum_optimizer})
        train_IOU += step_iou * X_batch.shape[0]
        train_IOU_no_classified += step_iou_no_classified * X_batch.shape[0]
        train_accuary += step_acc * X_batch.shape[0]
        train_loss += step_loss * X_batch.shape[0]
        train_summary_writer.add_summary(step_summary, step_value)

        percents = 100 * j / len(X_train)
        line_str = "Training: [{0}>{1}], {2}%, {3}/{4}, IOU:{5:.2f}, IOU_no:{6:.2f}, Accuracy:{7:.2f}, Loss:{8:.2f}, {9}s".format(
            "=" * (int(percents) // 2), " " * (50 - int(percents) // 2),
            int(percents), j, len(X_train), step_iou, step_iou_no_classified, step_acc, step_loss,
            int(time.time() - start_time))
        sys.stdout.write("\r" + line_str)
    train_IOU /= (len(X_train))
    train_IOU_no_classified /= (len(X_train))
    train_accuary /= (len(X_train))
    
    test_IOU_summary = sess.run(test_summary_IOU_op, feed_dict={test_IOU_input: train_IOU})
    train_summary_writer.add_summary(test_IOU_summary, i)
    
    test_IOU_no_classified_summary = sess.run(test_summary_IOU_op_no_classified, feed_dict={test_IOU_no_class_input: train_IOU_no_classified})
    train_summary_writer.add_summary(test_IOU_no_classified_summary, i)
    
    test_acc_summary = sess.run(test_summary_acc_op, feed_dict={test_acc_input: train_accuary})
    train_summary_writer.add_summary(test_acc_summary, i)
    
    test_loss_summary = sess.run(test_summary_loss_op, feed_dict={test_loss_input: train_loss})
    train_summary_writer.add_summary(test_loss_summary, i)
    
    print()
    print("EPOCH {0}, IOU: {1:.2f}, IOU_no:{2:.2f}, Accuracy: {3:.2f}, Loss: {4:.2f}".format(i, train_IOU, train_IOU_no_classified, train_accuary, train_loss))
    if i % 1 == 0:
        saver = tf.train.Saver()
        saver.save(sess, 'my_model/Dense_HED/', global_step=i)

Training: [=================================================> ], 99%, 48224/48238, IOU:0.53, IOU_no:0.48, Accuracy:0.98, Loss:0.00, 1726s
EPOCH 108, IOU: 0.64, IOU_no:0.58, Accuracy: 0.97, Loss: 141.31
Training: [=================================================> ], 99%, 48224/48238, IOU:0.69, IOU_no:0.58, Accuracy:0.98, Loss:0.01, 3526s
EPOCH 109, IOU: 0.65, IOU_no:0.58, Accuracy: 0.97, Loss: 142.87
Training: [=================================================> ], 99%, 48224/48238, IOU:0.77, IOU_no:0.65, Accuracy:1.00, Loss:0.00, 5332s
EPOCH 110, IOU: 0.64, IOU_no:0.57, Accuracy: 0.97, Loss: 150.48
Training: [=================================================> ], 99%, 48224/48238, IOU:0.68, IOU_no:0.59, Accuracy:0.98, Loss:0.00, 7130s
EPOCH 111, IOU: 0.65, IOU_no:0.59, Accuracy: 0.97, Loss: 140.67
Training: [=================================================> ], 99%, 48224/48238, IOU:0.60, IOU_no:0.58, Accuracy:0.91, Loss:0.00, 8932s
EPOCH 112, IOU: 0.65, IOU_no:0.59, Accuracy: 0.98, Los

KeyboardInterrupt: 

In [4]:
saver = tf.train.Saver()
saver.restore(sess, "my_model/Dense_HED/-121")

INFO:tensorflow:Restoring parameters from my_model/Dense_HED/-121


## Predict New Data

In [5]:
def generate_test_X(img):
    test_X = []
    for i in range(16, img.shape[0], 40):
        if i + 96 > img.shape[0]:
            continue
        for j in range(16, img.shape[1], 40):
            if j + 96 > img.shape[0]:
                continue
            test_X.append(img[i-16:i+80+16, j-16:j+80+16, :])
    test_X = np.asarray(test_X)
    return test_X

In [6]:
def generate_test_label(labels):
    test_label = np.zeros((200, 200, 1))
    for i in range(4):
        for j in range(4):
            test_label[i*40:i*40+80, j*40:j*40+80, :] += labels[i*4+j, 16:-16, 16:-16, :]
    return test_label

In [7]:
new_data = np.load('../new_data/new_data.npy', 'r')
new_data.shape

(3000, 15106, 8)

In [8]:
new_data = np.concatenate([new_data[:56,:,:],new_data,new_data[-232:,:,:]],axis=0)
new_data = np.concatenate([new_data[:,:56,:],new_data,new_data[:,-232:,:]],axis=1)
new_data.shape

(3288, 15394, 8)

In [9]:
i = 0
j = 0
result = np.zeros((new_data.shape[0], new_data.shape[1], 1))
from utils import ProgressBar
pb = ProgressBar(worksum=new_data.shape[0])
pb.startjob()
for i in range(0,new_data.shape[0],160):
    if i + 232 > new_data.shape[0]:
        continue
    for j in range(0,new_data.shape[1],160):
        if j + 232 > new_data.shape[1]:
            continue
        one_x = new_data[i:i + 232,j:j + 232,:]
        one_xes = generate_test_X(one_x)
        one_results = sess.run(pred, feed_dict={X_layer: one_xes, trainning_mode: False})
        one_result = generate_test_label(one_results)
        result[i+16:i + 200+16,j+16:j + 200+16, :] += one_result
    pb.complete(160)

 97.32 % [================================================>--] 3200/3288 	 used:358s eta:9 s

In [10]:
result = result[56:-232,56:-232] / 4
result.shape

(3000, 15106, 1)

In [11]:
np.save('result/Dense', result)

In [12]:
new_data = np.load('../new_data/new_data_0.npy', 'r')
new_data.shape

(3000, 15106, 8)

In [ ]:
new_data = np.concatenate([new_data[:56,:,:],new_data,new_data[-232:,:,:]],axis=0)
new_data = np.concatenate([new_data[:,:56,:],new_data,new_data[:,-232:,:]],axis=1)
new_data.shape

(3288, 15394, 8)

In [ ]:
i = 0
j = 0
result = np.zeros((new_data.shape[0], new_data.shape[1], 1))
from utils import ProgressBar
pb = ProgressBar(worksum=new_data.shape[0])
pb.startjob()
for i in range(0,new_data.shape[0],160):
    if i + 232 > new_data.shape[0]:
        continue
    for j in range(0,new_data.shape[1],160):
        if j + 232 > new_data.shape[1]:
            continue
        one_x = new_data[i:i + 232,j:j + 232,:]
        one_xes = generate_test_X(one_x)
        one_results = sess.run(pred, feed_dict={X_layer: one_xes, trainning_mode: False})
        one_result = generate_test_label(one_results)
        result[i+16:i + 200+16,j+16:j + 200+16, :] += one_result
    pb.complete(160)

 4.87 % [==>------------------------------------------------] 160/3288 	 used:17s eta:346 s

In [ ]:
result = result[56:-232,56:-232] / 4
result.shape

In [ ]:
result = cv2.flip(result, 0).reshape(result.shape[:2] + (1,))
result.shape

In [ ]:
np.save('result/Dense_0', result)